# Análise de Cesta 

### 🎲 Utilizando o algoritmo "A Priori" de mineração de dados

### 🔣 Dicionário de Dados

**Legenda das Colunas da Tabela de Regras de Associação**

* **`antecedents`**:
    * O(s) código(s) do(s) produto(s) no lado esquerdo da regra (item X em "SE compra X...").
    * Tipo: Texto (object) ou Número (int64), idealmente tratado como Texto.

* **`consequents`**:
    * O(s) código(s) do(s) produto(s) no lado direito da regra (item Y em "...ENTÃO compra Y").
    * Tipo: Texto (object) ou Número (int64), idealmente tratado como Texto.

* **`antecedent support`**:
    * Frequência/proporção de transações que contêm o antecedente (X). `P(X)`.
    * Tipo: Número Decimal (float64). *Formatar como Percentual (%) para visualização.*

* **`consequent support`**:
    * Frequência/proporção de transações que contêm o consequente (Y). `P(Y)`.
    * Tipo: Número Decimal (float64). *Formatar como Percentual (%) para visualização.*

* **`support`**:
    * Frequência/proporção de transações que contêm *tanto* o antecedente (X) *quanto* o consequente (Y). `P(X e Y)`.
    * Indica a popularidade geral da *combinação* X e Y.
    * Tipo: Número Decimal (float64). *Formatar como Percentual (%) para visualização.*

* **`confidence`**:
    * Probabilidade de encontrar o consequente (Y) em uma transação, *dado que* ela já contém o antecedente (X). `P(Y | X)`.
    * Mede a *confiabilidade* ou força preditiva da regra na direção X -> Y.
    * Tipo: Número Decimal (float64). *Formatar como Percentual (%) para visualização.*

* **`lift`**:
    * Mede quantas vezes a presença de X *aumenta* a probabilidade de comprar Y, comparado à probabilidade normal de comprar Y. `Conf(X->Y) / Sup(Y)`.
    * Lift > 1 indica associação positiva (interessante); Lift = 1 indica independência; Lift < 1 indica associação negativa.
    * Tipo: Número Decimal (float64). *Formatar como Número Decimal (ex: 2 casas).*

* **`representativity`**: (Atenção: Métrica menos padronizada, pode ter definições variadas)
    * Tenta medir quão "representativa" é a regra ou a dependência encontrada, possivelmente comparando métricas observadas com esperadas.
    * *Verificar a documentação da biblioteca usada ou a fórmula específica para interpretação precisa.*
    * Tipo: Número Decimal (float64). *Formatar como Número Decimal.*

* **`leverage`**:
    * Mede a diferença entre a frequência observada de X e Y juntos e a frequência esperada se fossem independentes. `Sup(X,Y) - Sup(X)*Sup(Y)`.
    * Valores > 0 indicam que aparecem juntos mais que o esperado.
    * Tipo: Número Decimal (float64). *Formatar como Número Decimal.*

* **`conviction`**:
    * Mede o grau de "implicação" da regra. Alto valor sugere que o consequente depende fortemente do antecedente. `(1-Sup(Y))/(1-Conf(X->Y))`.
    * Pode ser muito alto ou infinito.
    * Tipo: Número Decimal (float64). *Formatar como Número Decimal.*

* **`zhangs_metric`**:
    * Métrica robusta (varia de -1 a +1) que mede a associação levando em conta suporte e confiança, sem ser afetada pela popularidade isolada.
    * Valores próximos de +1 indicam forte associação positiva.
    * Tipo: Número Decimal (float64). *Formatar como Número Decimal.*

* **`jaccard`**:
    * Coeficiente de Jaccard: similaridade entre os conjuntos de transações contendo X e Y. `Sup(X,Y) / (Sup(X)+Sup(Y)-Sup(X,Y))`.
    * Valor entre 0 e 1 (pode ser visto como %).
    * Tipo: Número Decimal (float64). *Formatar como Número Decimal ou Percentual (%).*

* **`certainty`**:
    * Mede o ganho de "certeza" proporcionado pela regra. `(Conf(X->Y) - Sup(Y)) / (1 - Sup(Y))`.
    * Varia de -1 a 1.
    * Tipo: Número Decimal (float64). *Formatar como Número Decimal.*

* **`kulczynski`**:
    * Média das confianças nas duas direções: `0.5 * (Conf(X->Y) + Conf(Y->X))`.
    * Valor entre 0 e 1 (pode ser visto como %).
    * Tipo: Número Decimal (float64). *Formatar como Número Decimal ou Percentual (%).*

* **`Nomes_Antecedents`**:
    * O(s) nome(s) legível(is) do(s) produto(s) antecedente(s).
    * Tipo: Texto (object).

* **`Nomes_Consequents`**:
    * O(s) nome(s) legível(is) do(s) produto(s) consequente(s).
    * Tipo: Texto (object).


### 📚 Packages

In [1]:
import polars as pl
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx


### 🧩 E.D.A
- Dados extraídos do Kaggle - [Brazilian E-Commerce Public Dataset by Olist](https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce)

In [2]:
orders = pl.read_csv("../data/olist_orders_dataset.csv")
order_items = pl.read_csv("../data/olist_order_items_dataset.csv")
products = pl.read_csv("../data/olist_products_dataset.csv")

In [7]:
orders.head()

order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
str,str,str,str,str,str,str,str
"""e481f51cbdc54678b7cc49136f2d6a…","""9ef432eb6251297304e76186b10a92…","""delivered""","""2017-10-02 10:56:33""","""2017-10-02 11:07:15""","""2017-10-04 19:55:00""","""2017-10-10 21:25:13""","""2017-10-18 00:00:00"""
"""53cdb2fc8bc7dce0b6741e21502734…","""b0830fb4747a6c6d20dea0b8c802d7…","""delivered""","""2018-07-24 20:41:37""","""2018-07-26 03:24:27""","""2018-07-26 14:31:00""","""2018-08-07 15:27:45""","""2018-08-13 00:00:00"""
"""47770eb9100c2d0c44946d9cf07ec6…","""41ce2a54c0b03bf3443c3d931a3670…","""delivered""","""2018-08-08 08:38:49""","""2018-08-08 08:55:23""","""2018-08-08 13:50:00""","""2018-08-17 18:06:29""","""2018-09-04 00:00:00"""
"""949d5b44dbf5de918fe9c16f97b45f…","""f88197465ea7920adcdbec7375364d…","""delivered""","""2017-11-18 19:28:06""","""2017-11-18 19:45:59""","""2017-11-22 13:39:59""","""2017-12-02 00:28:42""","""2017-12-15 00:00:00"""
"""ad21c59c0840e6cb83a9ceb5573f81…","""8ab97904e6daea8866dbdbc4fb7aad…","""delivered""","""2018-02-13 21:18:39""","""2018-02-13 22:20:29""","""2018-02-14 19:46:34""","""2018-02-16 18:17:02""","""2018-02-26 00:00:00"""


In [8]:
order_items.head()

order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
str,i64,str,str,str,f64,f64
"""00010242fe8c5a6d1ba2dd792cb162…",1,"""4244733e06e7ecb4970a6e2683c13e…","""48436dade18ac8b2bce089ec2a0412…","""2017-09-19 09:45:35""",58.9,13.29
"""00018f77f2f0320c557190d7a144bd…",1,"""e5f2d52b802189ee658865ca93d83a…","""dd7ddc04e1b6c2c614352b383efe2d…","""2017-05-03 11:05:13""",239.9,19.93
"""000229ec398224ef6ca0657da4fc70…",1,"""c777355d18b72b67abbeef9df44fd0…","""5b51032eddd242adc84c38acab88f2…","""2018-01-18 14:48:30""",199.0,17.87
"""00024acbcdf0a6daa1e931b038114c…",1,"""7634da152a4610f1595efa32f14722…","""9d7a1d34a5052409006425275ba1c2…","""2018-08-15 10:10:18""",12.99,12.79
"""00042b26cf59d7ce69dfabb4e55b4f…",1,"""ac6c3623068f30de03045865e4e100…","""df560393f3a51e74553ab94004ba5c…","""2017-02-13 13:57:51""",199.9,18.14


In [9]:
products.head()

product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
str,str,i64,i64,i64,i64,i64,i64,i64
"""1e9e8ef04dbcff4541ed26657ea517…","""perfumaria""",40,287,1,225,16,10,14
"""3aa071139cb16b67ca9e5dea641aaa…","""artes""",44,276,1,1000,30,18,20
"""96bd76ec8810374ed1b65e29197571…","""esporte_lazer""",46,250,1,154,18,9,15
"""cef67bcfe19066a932b7673e239eb2…","""bebes""",27,261,1,371,26,4,26
"""9dc1a7de274444849c219cff195d0b…","""utilidades_domesticas""",37,402,4,625,20,17,13


- Shape de cada DF

In [11]:
print(f"\n Ordem: {orders.shape}\n Ordem Produtos: {orders.shape} \n Produtos: {orders.shape}")


 Ordem: (99441, 8)
 Ordem Produtos: (99441, 8) 
 Produtos: (99441, 8)


- Verificando valores NaN

In [ ]:
def reportar_nulos_polars(df: pl.DataFrame, nome_df: str):
    """
    Verifica valores nulos em um DataFrame Polars e reporta as colunas com nulos.
    USA .unpivot() e calcula o total corretamente.

    Args:
        df: O DataFrame Polars a ser verificado.
        nome_df: Uma string com o nome do DataFrame para identificação na saída.
    """
    print(f"\n--- Verificando Nulos em '{nome_df}' ---")

    # 1. Contar nulos em cada coluna
    df_null_counts = df.null_count()

    # 2. Transformar para formato longo e filtrar colunas com nulos > 0
    df_null_long = df_null_counts.unpivot() # Colunas: 'variable', 'value'
    df_null_filtered = df_null_long.filter(pl.col("value") > 0)

    # 3. Imprimir as colunas filtradas
    print("Colunas com valores nulos e suas contagens:")
    if df_null_filtered.height == 0:
        print("Nenhuma coluna com valores nulos encontrada.")
    else:
        for row_dict in df_null_filtered.iter_rows(named=True):
            coluna = row_dict['variable']
            contagem = row_dict['value']
            print(f"- {coluna}: {contagem}")

    # --- CORREÇÃO AQUI ---
    # 4. Calcular e imprimir o total de nulos
    # Soma a coluna 'value' do DataFrame longo (df_null_long)
    total_nulos = df_null_long['value'].sum()
    # --- FIM DA CORREÇÃO ---

    print(f"\nTotal de valores nulos no DataFrame '{nome_df}': {total_nulos}")
    print("-" * (len(f"--- Verificando Nulos em '{nome_df}' ---") + 4))


# --- Chamar a função para cada DataFrame (como antes) ---
print("Iniciando verificação geral de valores nulos...")

dataframes_para_verificar = {
    "orders": orders,
    "order_items": order_items,
    "products": products
}

for nome, df_atual in dataframes_para_verificar.items():
    reportar_nulos_polars(df_atual, nome)


Iniciando verificação geral de valores nulos...

--- Verificando Nulos em 'orders' ---
Colunas com valores nulos e suas contagens:
- order_approved_at: 160
- order_delivered_carrier_date: 1783
- order_delivered_customer_date: 2965

Total de valores nulos no DataFrame 'orders': 4908
-----------------------------------------

--- Verificando Nulos em 'order_items' ---
Colunas com valores nulos e suas contagens:
Nenhuma coluna com valores nulos encontrada.

Total de valores nulos no DataFrame 'order_items': 0
----------------------------------------------

--- Verificando Nulos em 'products' ---
Colunas com valores nulos e suas contagens:
- product_category_name: 610
- product_name_lenght: 610
- product_description_lenght: 610
- product_photos_qty: 610
- product_weight_g: 2
- product_length_cm: 2
- product_height_cm: 2
- product_width_cm: 2

Total de valores nulos no DataFrame 'products': 2448
-------------------------------------------

Verificação de nulos concluída.


### Separando os dados e agrupando por transação

In [3]:
# Juntando pedidos com itens
order_details = order_items.join(orders, on='order_id', how='inner')

In [4]:
# Filtrando pedidos validos
status_validos = ['delivered']
order_details_filtered = order_details.filter(pl.col('order_status').is_in(status_validos))

In [5]:
# Selecionando colunas chaves
baskets_raw = order_details_filtered.select(['order_id', 'product_id'])

In [6]:
# Garantindo tipos corretos
baskets_typed = baskets_raw.with_columns(
    pl.col('product_id').cast(pl.Utf8)
)

# Removendo duplicatas por transaçao
basket_items_unique = baskets_typed.unique(subset=['order_id', 'product_id'], keep='first')

- Agrupamento dse produtos por pedida que não teve sucesso no Polars, vou aplicar Pandas

Analise o motivo do erro...

In [8]:
# Agrupando produtos por pedido (Transação)

baskets_grouped = basket_items_unique.group_by('order_id').agg(
    pl.col('product_id').list().alias('products')
)

TypeError: 'ExprListNameSpace' object is not callable

In [9]:
import polars as pl
print(pl.__version__)
df_teste = pl.DataFrame({'grupo': ['A', 'A', 'B'], 'item': ['x1', 'x2', 'y1']})
try:
    df_agrupado_teste = df_teste.group_by('grupo').agg(pl.col('item').list().alias('lista_itens'))
    print("SUCESSO no ambiente limpo!")
    print(df_agrupado_teste)
except Exception as e:
    print(f"ERRO no ambiente limpo: {e}")

0.20.18
ERRO no ambiente limpo: 'ExprListNameSpace' object is not callable


- Agrupamento usando Pandas (temporário)

In [10]:
transactions_list = None # Inicializar para o caso de erro
try:
    # Converter o DataFrame Polars para Pandas
    basket_items_unique_pd = basket_items_unique.to_pandas()
    print(f"Convertido para Pandas. Shape: {basket_items_unique_pd.shape}")

    # Aplicar o groupby do Pandas para obter a lista de listas
    # Usando a sintaxe que você confirmou que funciona
    transactions_list = basket_items_unique_pd.groupby('order_id')['product_id'].apply(list).tolist()

    print(f"Passo 6: Agrupamento com Pandas concluído. Número de cestas: {len(transactions_list)}")
    # print(transactions_list[:5]) # Descomente para verificar as primeiras listas

except Exception as e:
    print(f"Erro durante o Passo 6 (Conversão/Agrupamento Pandas): {e}")

Convertido para Pandas. Shape: (100196, 2)
Passo 6: Agrupamento com Pandas concluído. Número de cestas: 96478


### 📈 Aplicando TransactionEncoder e apriori:

    Vamos transformar os dados e encontrar os itemsets frequentes e as regras de associação usando mlxtend.

#### Aplicando o TransactionEncoder: 
- Transforma a lista de listas 'transacoes' em formato one-hot encoded, isto é, cada coluna é um produto e cada linha é uma transação. 

In [11]:
if transactions_list is not None:
    # 7. Preparar Lista para TransactionEncoder (Já temos 'transactions_list')
    print("\nPasso 7: Lista de transações pronta.")

    # 8. Aplicar TransactionEncoder e Criar DataFrame Pandas (como antes)
    print("Passo 8: Aplicando TransactionEncoder e criando DataFrame one-hot encoded (Pandas)...")
    te = TransactionEncoder()
    te_ary = te.fit(transactions_list).transform(transactions_list)
    # O resultado df_encoded_pd será um DataFrame Pandas
    df_encoded_pd = pd.DataFrame(te_ary, columns=te.columns_)

    print("\n--- Preparação Concluída ---")
    print("DataFrame 'df_encoded_pd' (Pandas) pronto para a função apriori do mlxtend.")
    print(f"Shape: {df_encoded_pd.shape}")
    # print(df_encoded_pd.head())

else:
    print("\nNão foi possível continuar devido a erro no passo 6.")


Passo 7: Lista de transações pronta.
Passo 8: Aplicando TransactionEncoder e criando DataFrame one-hot encoded (Pandas)...

--- Preparação Concluída ---
DataFrame 'df_encoded_pd' (Pandas) pronto para a função apriori do mlxtend.
Shape: (96478, 32216)


In [12]:
print(df_encoded_pd.head())

   00066f42aeeb9f3007548bb9d3f33c38  00088930e925c41fd95ebfe695fd2655  \
0                             False                             False   
1                             False                             False   
2                             False                             False   
3                             False                             False   
4                             False                             False   

   0009406fd7479715e4bef61dd91f2462  000b8f95fcb9e0096488278317764d19  \
0                             False                             False   
1                             False                             False   
2                             False                             False   
3                             False                             False   
4                             False                             False   

   000d9be29b5207b54e86aa1b1ac54872  0011c512eb256aa0dbbb544d8dffcf6e  \
0                             False              

#### Aplicando o Algoritmo Apriori: 
- Encontrar o conjunto de itens, vulgo 'itemset' que aparecem juntos com frequência;
- O 'min_support' define a frequência mínima para um itemset ser considerado frequente;
Ex: min_support=0.01 significa que o itemset deve aparecer em pelo menos 1% das transações;

**Ajuste 'min_support' conforme necessário!** Comece com um valor pequeno (ex: 0.01 ou 0.005)

In [13]:
# frequent_itemsets é um DataFrame com os itemsets frequentes e seus suportes
min_support_valor = 0.001
print(f"\nExecutando apriori com min_support = {min_support_valor} \n")

frequent_itemsets = apriori(df_encoded_pd, min_support=min_support_valor, use_colnames=True)
print(f"Encontrados {frequent_itemsets.shape[0]} itemsets frequentes.")


Executando apriori com min_support = 0.001 

Encontrados 52 itemsets frequentes.


#### Gerando regras de associação: 
- Cria as regras (ex: {Produto A} => {Produto B}) a partir dos itemsets frequentes.
- Usamos uma métrica ('lift', 'confidence', 'support') e um limiar mínimo ('min_threshold').
- 'lift' > 1 sugere que os itens aparecem juntos mais do que o esperado por acaso.
- 'confidence' mede a probabilidade de comprar B dado que A foi comprado.

**Ajuste a métrica e o 'min_threshold' conforme necessário!**

In [ ]:
metric = "lift" # Ou 'confidence'

min_threshold_valor = 1 # Ajuste (para lift > 1, para confidence > 0.1, etc.)
print(f"\nGerando regras com {metric} >= {min_threshold_valor}...")

rules = association_rules(frequent_itemsets, metric=metric, min_threshold=min_threshold_valor)
print("Regras geradas.")
print(f"Encontradas {rules.shape[0]} regras.")


Gerando regras com lift >= 1...
Regras geradas.
Encontradas 0 regras.


In [15]:
# Adicionar uma coluna com o tamanho de cada itemset
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
print(frequent_itemsets.head()) # Ver início
print(frequent_itemsets.tail()) # Ver fim
print(f"\nTamanho máximo de itemset encontrado: {frequent_itemsets['length'].max()}")
print(f"Número de itemsets com mais de 1 item: {len(frequent_itemsets[frequent_itemsets['length'] >= 2])}")

    support                            itemsets  length
0  0.001327  (06edb72f1e0c64b14c5b79353f7abea3)       1
1  0.001420  (0aabfb375647d9738ad0f7b4ea3653b1)       1
2  0.001037  (0bcc3eeca39e1064258aa1e932269894)       1
3  0.002716  (154e7e31ebfa092203795c972e5804a6)       1
4  0.001078  (165f86fe8b799a708a20ee4ba125c289)       1
     support                            itemsets  length
47  0.001358  (e0cf79767c5b016251fe139915c59a26)       1
48  0.002000  (e0d64dcfaa3b6db5c54ca298ae101d05)       1
49  0.001596  (e53e557d5a159f5aa2c5e995dfdf244b)       1
50  0.001202  (ec2d43cc59763ec91694573b31f1c29a)       1
51  0.001544  (f1c7f353075ce59d8a6f3cf58f419c9c)       1

Tamanho máximo de itemset encontrado: 1
Número de itemsets com mais de 1 item: 0
